In [1]:
import os
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
chat = ChatOpenAI(
    openai_api_key=openai_api_key,
    model='gpt-3.5-turbo'
)

In [2]:
from langchain.schema import (
    SystemMessage,
    HumanMessage
)

def get_user_msg(user_msg: str):
    messages = [
        SystemMessage(content='Ты умный ассистент, отвечающий на вопросы пользователя о файлах, содержащихся в базе данных. Полезные для ответа части текста из файлов будут поданы в контексте. Используй их, чтобы дать точный и полный ответ на вопрос пользователя. Не матерись. Все матерные слова заменяй на звездочки.'),
        HumanMessage(content=user_msg)
    ]
    return chat(messages).content

get_user_msg("Привет, расскажи про LLM")

C:\Users\Борис\PycharmProjects\pro_test\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Привет! LLM расшифровывается как Long Short-Term Memory, что является типом рекуррентной нейронной сети. Она используется для обработки последовательных данных, сохранения долгосрочных зависимостей и предотвращения проблемы затухающих градиентов. LLM имеет способность запоминать информацию на протяжении длительного временного интервала, что делает ее эффективной для работы с различными задачами, такими как анализ текста, машинный перевод, распознавание речи и другие.'

In [3]:
from langchain.document_loaders import DirectoryLoader
DATA_PATH = 'data'
def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob='*.md')
    documents = loader.load()
    return documents

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
def split_text(documents: list):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=500,
        length_function=len,
        add_start_index=True
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")
    document = chunks[10]
    print(document.page_content)
    print(document.metadata)
    return chunks

In [5]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
import shutil


CHROMA_PATH = "chroma"


def save_to_chroma(chunks: list):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    db = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [6]:
def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)
generate_data_store()

Split 1 documents into 102 chunks.
Формат: видео

Пик популярности: январь

Где расходился: «Тикток», «Инстаграм»*

С премьеры «Образцового самца» прошло более 20 лет, а мемный потенциал картины в полном объеме раскрыли только в 2023 году, когда дуэль взглядов актеров Бена Стиллера и Оуэна Уилсона сделали мемным шаблоном. В мемах Стиллер олицетворяет ничего не подозревающего человека, который был счастлив, пока не столкнулся с Уилсоном — неким препятствием или разочаровывающим обстоятельством.
{'source': 'data\\memes.md', 'start_index': 2145}
Saved 102 chunks to chroma.


In [7]:
from langchain_core.prompts import ChatPromptTemplate
import argparse
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
def main(query_text):

    # Prepare the DB.
    embedding_function = OpenAIEmbeddings()
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=3)
    if len(results) == 0 or results[0][1] < 0.7:
        print(f"Unable to find matching results.")
        return

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    print(prompt)

    model = ChatOpenAI()
    response_text = model.predict(prompt)

    sources = [doc.metadata.get("source", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

In [8]:
main('Кто такие Сигмы?')

Human: 
Answer the question based only on the following context:

Формат: мультиформатный

Пик популярности: зима, но вообще — весь год

Где расходился: «Тикток», «Инстаграм»*, «Ютуб»

Сигмами называют крутых парней — главных героев «Бойцовского клуба», «Острых козырьков» или «Джона Уика». И тех, кто очень хочет быть на них похожим и видит в персонажах буквально себя. Смысл термина постоянно трансформируется. Сигмы и одинокие волки, и сверхуспешные трудоголики, и образцы маскулинности, которые с уважением относятся к окружающим.

---

Хотя сигмы обсуждались и до 2023, в этом году мы как-то особенно часто видели людей, которые особым образом хмурили брови и надували губы, выражая одобрение происходящему. А списана гримаса как раз с одного из главных сигм поп-культуры — Патрика Бейтмана из «Американского психопата».

«Образовый самец» оборачивается

За что мы любим мем: безумные взгляды легендарных персонажей

Формат: видео

Пик популярности: январь

Где расходился: «Тикток», «Инстаграм»

C:\Users\Борис\PycharmProjects\pro_test\venv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response: Сигмы - это крутые парни, главные герои из различных фильмов и сериалов, которых люди хотят быть похожими и видят себя в них. Сигмы также представляют собой образцы маскулинности, которые уважительно относятся к окружающим.
Sources: ['data\\memes.md', 'data\\memes.md', 'data\\memes.md']
